In [1]:
import numpy as np
import pickle
import project_path
import os
from model.neuron_metadata import *
from model.data_accessor import get_data_file_abs_path
from model.neural_model import NeuralModel
from util.plot_util import *
from scipy import optimize
from numpy import linalg as LA
import sympy as sp
import time

neuron_metadata_collection = NeuronMetadataCollection.load_from_chem_json(get_data_file_abs_path('chem.json'))
N = neuron_metadata_collection.get_size()
neuron_to_stimulate = "PLMR"

In [3]:
model = NeuralModel(neuron_metadata_collection)
model.init_kunert_2017()

# Get standard equilibrium
I_nA = 1.4
cur_I_ext = np.zeros(N)
neuron_id = neuron_metadata_collection.get_id_from_name(neuron_to_stimulate)
cur_I_ext[neuron_id] = I_nA * 10000
model.cur_I_ext = cur_I_ext
model.set_I_ext_constant_currents({neuron_to_stimulate: I_nA})
standard_equi = model.compute_standard_equilibrium()

In [4]:
def get_dynamics_symbols(model):
  """
  Get the dV/dt and dS/dt in symbols, so you can do things like substitutions / computing jacobians.
  """
  N = model.N
  vsymbols = sp.symbols(' '.join([('v'+str(i)) for i in range(N)]))
  ssymbols = sp.symbols(' '.join([('s'+str(i)) for i in range(N)]))
  vs = sp.Matrix(vsymbols)
  ss = sp.Matrix(ssymbols)

  Gs_sp = sp.Matrix(model.Gs)
  Gg_sp = sp.Matrix(model.Gg)
  E_sp = sp.Matrix(model.E)
  Gg_squashed_col_sp = sp.Matrix(model.Gg.sum(axis = 1))
  neg_Ec_sp = sp.ones(N,1) * (-model.Ec)

  # I_leak
  I_leak = model.Gc * (vs + neg_Ec_sp)

  # I_gap = sum_j G_ij (V_i - V_j) = V_i sum_j G_ij - sum_j G_ij V_j
  # The first term is a point-wise multiplication of V and G's squashed column.
  # The second term is matrix multiplication of G and V
  I_gap = sp.matrix_multiply_elementwise(Gg_squashed_col_sp, vs) - Gg_sp * vs

  # I_syn = sum_j G_ij s_j (V_i - E_j) = V_i sum_j G_ij s_j - sum_j G_j s_j E_j
  # First term is a point-wise multiplication of V and (Matrix mult of G and s)
  # Second term is matrix mult of G and (point mult of s_j and E_j)
  I_syn = sp.matrix_multiply_elementwise(vs, Gs_sp * ss) -\
      Gs_sp * sp.matrix_multiply_elementwise(ss, E_sp)

  dV_dts = (-I_leak - I_gap - I_syn + sp.Matrix(model.cur_I_ext)) / model.C
  
  v_min_vths = vs - sp.Matrix(model.Vth)
  phis = v_min_vths.applyfunc(lambda v_min_vth: 1.0 / (1.0 + sp.exp(-model.B * v_min_vth)))
  syn_rises = model.ar * sp.matrix_multiply_elementwise(phis, (sp.ones(N,1) - ss))
  syn_drops = model.ad * ss
  dS_dts = syn_rises - syn_drops
  return vsymbols, ssymbols, sp.Matrix([dV_dts, dS_dts])

In [5]:
start_time = time.time()
vsymbols, ssymbols, dyn_symbols = get_dynamics_symbols(model)
print("Computing symbols takes %.2fs" % (time.time() - start_time))

Computing symbols takes 9.02s


In [6]:
# This shows that our symbolic dynamics calculation matches that from the original model
# We did our symbols correctly!

# Evaluated dV/dt at standard equilibrium
test_state_vars_1 = list(range(2*N))
test_state_vars = test_state_vars_1
sub_dict = {vsymbols[i]: test_state_vars[i] for i in range(N)}
sub_dict.update({ssymbols[i]: test_state_vars[i+N] for i in range(N)})

dyns_from_symbol = dyn_symbols.xreplace(sub_dict)
dyns_from_model = model.dynamic(t=0, state_vars = np.array(test_state_vars))
print(dyns_from_symbol[0:5])
print(dyns_from_model[0:5])
print(dyns_from_symbol[-5:])
print(dyns_from_model[-5:])
print(np.mean(np.abs(np.array(list(dyns_from_symbol)) - np.array(dyns_from_model))))

[-350.000000000000, -30460.0000000000, -349370.000000000, -743180.000000000, -1831790.00000000]
[-3.50000e+02 -3.04600e+04 -3.49370e+05 -7.43180e+05 -1.83179e+06]
[-3316.99999999015, -3323.00000000000, -2775.00000000000, -3335.00000000000, -3340.98499606003]
[-3316.99999999 -3323.         -2775.         -3335.
 -3340.98499606]
2.19168012133903e-9


In [7]:
start_time = time.time()
all_symbols = list(vsymbols) + list(ssymbols)
J = sp.Matrix(dyn_symbols).jacobian(all_symbols)
print("Computing symbolic jacobian takes %.2fs" % (time.time() - start_time))

Computing symbolic jacobian takes 27.90s


# Compute Jacobian eigenvalues as PLMR input is varied

Note that we just keep using the same symbolic jacobian for all input. This is because I_ext will get differentiated away in the jacobian matrix. The only thing that changes with I_nA is the standard equilibrium.

From manual testing, somewhere between 1.75 and 1.85 nA, we get Hopf bifurcation.

In [8]:
print("I_nA, max_real_eval")
for I_nA in np.arange(0.0,3.0,0.1):
  model = NeuralModel(neuron_metadata_collection)
  model.init_kunert_2017()

  # Get standard equilibrium
  cur_I_ext = np.zeros(N)
  neuron_id = neuron_metadata_collection.get_id_from_name(neuron_to_stimulate)
  cur_I_ext[neuron_id] = I_nA * 10000
  model.cur_I_ext = cur_I_ext
  model.set_I_ext_constant_currents({neuron_to_stimulate: I_nA})
  standard_equi = model.compute_standard_equilibrium()
  
  sub_dict = {vsymbols[i]: standard_equi[i] for i in range(N)}
  sub_dict.update({ssymbols[i]: standard_equi[i+N] for i in range(N)})
  start_time = time.time()
  evaluated_j = J.xreplace(sub_dict)
  print("Evaluating jacobian takes %.2fs" % (time.time() - start_time))
  j_np = np.array(evaluated_j).astype(np.float64)
  evals = LA.eigvals(j_np)
  print("%.2f, %.2f" % (I_nA, max(evals)))

I_nA, max_real_eval
Evaluating jacobian takes 0.91s


/home/stephen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: ComplexWarning: Casting complex values to real discards the imaginary part


0.00, -4.54
Evaluating jacobian takes 0.86s
0.10, -4.57
Evaluating jacobian takes 0.87s
0.20, -4.60
Evaluating jacobian takes 0.89s
0.30, -4.63
Evaluating jacobian takes 0.87s
0.40, -4.66
Evaluating jacobian takes 1.02s
0.50, -4.68
Evaluating jacobian takes 0.86s
0.60, -4.71
Evaluating jacobian takes 0.86s
0.70, -4.74
Evaluating jacobian takes 0.89s
0.80, -4.77
Evaluating jacobian takes 0.89s
0.90, -4.80
Evaluating jacobian takes 0.84s
1.00, -4.83
Evaluating jacobian takes 0.92s
1.10, -4.86
Evaluating jacobian takes 0.85s
1.20, -4.61
Evaluating jacobian takes 0.85s
1.30, -3.92
Evaluating jacobian takes 0.82s
1.40, -1.24
Evaluating jacobian takes 0.96s
1.50, -4.14
Evaluating jacobian takes 0.83s
1.60, -4.89
Evaluating jacobian takes 0.84s
1.70, -4.89
Evaluating jacobian takes 0.83s
1.80, -4.89
Evaluating jacobian takes 0.84s
1.90, -4.89
Evaluating jacobian takes 0.84s
2.00, -4.89
Evaluating jacobian takes 0.83s
2.10, -4.89
Evaluating jacobian takes 1.00s
2.20, -4.90
Evaluating jacobian 